# Assignment 5 - Text Analysis
An explanation this assignment could be found in the .pdf explanation document


## Materials to review for this assignment
<h4>From Moodle:</h4> 
<h5><u>Review the notebooks regarding the following python topics</u>:</h5>
<div class="alert alert-info">
&#x2714; <b>Working with strings</b> (tutorial notebook)<br/>
&#x2714; <b>Text Analysis</b> (tutorial notebook)<br/>
&#x2714; <b>Hebrew text analysis tools (tokenizer, wordnet)</b> (moodle example)<br/>
&#x2714; <b>(brief review) All previous notebooks</b><br/>
</div> 
<h5><u>Review the presentations regarding the following topics</u>:</h5>
<div class="alert alert-info">
&#x2714; <b>Text Analysis</b> (lecture presentation)<br/>
&#x2714; <b>(brief review) All other presentations</b><br/>
</div>

## Preceding Step - import modules (packages)
This step is necessary in order to use external modules (packages). <br/>

In [66]:
# --------------------------------------
import pandas as pd
import numpy as np
# --------------------------------------


# --------------------------------------
# ------------- visualizations:
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
# --------------------------------------


# ---------------------------------------
import sklearn
from sklearn import preprocessing, metrics, pipeline, model_selection, feature_extraction 
from sklearn import naive_bayes, linear_model, svm, neural_network, neighbors, tree
from sklearn import decomposition, cluster

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV 
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import mean_squared_error, r2_score, silhouette_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder

from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import Perceptron, SGDClassifier
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
# ---------------------------------------


# ----------------- output and visualizations: 
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.simplefilter("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter("ignore", category=ConvergenceWarning)
# show several prints in one cell. This will allow us to condence every trick in one cell.
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline
pd.pandas.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
# ---------------------------------------

### Text analysis and String manipulation imports:

In [67]:
# --------------------------------------
# --------- Text analysis and Hebrew text analysis imports:
# vectorizers:
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# regular expressions:
import re
# --------------------------------------

### (optional) Hebrew text analysis - WordNet (for Hebrew)
Note: the WordNet is not a must

#### (optional) Only if you didn't install Wordnet (for Hebrew) use:

In [68]:
# word net installation:

# unmark if you want to use and need to install
#!pip install wn
# !python -m wn download omw-he:1.4

In [69]:
# word net import:

# unmark if you want to use:
 #import wn

### (optional) Hebrew text analysis - hebrew_tokenizer (Tokenizer for Hebrew)
Note: the hebrew_tokenizer is not a must

#### (optional) Only if you didn't install hebrew_tokenizer use:

In [70]:
# Hebrew tokenizer installation:

# unmark if you want to use and need to install:
#!pip install hebrew_tokenizer

In [71]:
# Hebrew tokenizer import:

# unmark if you want to use:
#import hebrew_tokenizer as ht

### Reading input files
Reading input files for train annotated corpus (raw text data) corpus and for the test corpus

In [72]:
train_filename = 'annotated_corpus_for_train.csv'
test_filename  = 'corpus_for_test.csv'
df_train = pd.read_csv(train_filename, index_col=None, encoding='utf-8')
df_test  = pd.read_csv(test_filename, index_col=None, encoding='utf-8')

In [73]:
df_train.head(8)
df_train.shape

,story,gender
0,"כשחבר הזמין אותי לחול, לא באמת חשבתי שזה יקרה,...",m
1,לפני שהתגייסתי לצבא עשיתי כל מני מיונים ליחידו...,m
2,מאז שהתחילו הלימודים חלומו של כל סטודנט זה הפנ...,f
3,"כשהייתי ילד, מטוסים היה הדבר שהכי ריתק אותי. ב...",m
4,‏הייתי מדריכה בכפר נוער ומתאם הכפר היינו צריכי...,f
5,לפני כ3 חודשים טסתי לרומא למשך שבוע. טסתי במטו...,f
6,אני כבר שנתיים נשוי והשנה אני ואישתי סוף סוף י...,m
7,השנה התחלנו שיפוץ בדירה שלנו בתל אביב. הדירה ה...,f


(753, 2)

In [74]:
df_test.head(3)
df_test.shape

,test_example_id,story
0,0,כל קיץ אני והמשפחה נוסעים לארצות הברית לוס אנג...
1,1,"הגעתי לשירות המדינה אחרי שנתיים כפעיל בתנועת ""..."
2,2,אחת האהבות הגדולות שלי אלו הכלבים שלי ושל אישת...


(323, 2)

### Your implementation:
Write your code solution in the following code-cells

In [75]:
# YOUR CODE HERE
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
import string
import re
from sklearn.svm import SVC
from sklearn.metrics import f1_score, make_scorer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.exceptions import NotFittedError


# Apply text preprocessing to the training data

Remove any non-Hebrew characters from the text data.

In [76]:
# YOUR CODE HERE
punctuations = string.punctuation + "׃׀׆׳״" 
df_train['story'] = df_train['story'].apply(lambda x: "".join([char for char in x if char not in punctuations]))
pattern = r'[^א-ת\s]|\d'
df_train['story'] =df_train['story'].apply(lambda x: re.sub(pattern, '', x))
print(df_train.iloc[5]['story'])

לפני כ חודשים טסתי לרומא למשך שבוע טסתי במטוס אל על הטיסה ארכה כ וחצי שעות לא היו עיכובים נחתנו בערך בשעה  בבוקר ברומא מהשדה תעופה נסענו למלון  משם הלכנו למזרקת טרווי  לפי אגדה מסורתית השלכת מטבע למזרקה ביד ימין מעבר לכתף שמאל מבטיחה את שובו של המשליך לרומא מעריכים כי מטבעות בשווי כשלושת אלפים אירו נגרפות מדי לילה מתחתית המזרקה הסתובבנו באזור של המזרקה לקראת הערב נסענו לארוחת ערב במלון  יום למחרת נסענו בבוקר לגני וילה בורגוזה  גן בסגנון אנגלי בגן מצויים מספר מבנים המשמשים כיום כמוזיאונים והוא הגן הציבורי השני בגודלו ברומא השכרנו אוטו גולף והיינו שם בערך כשעתיים משם המשכנו לפיאצה דל פופולו  כיכר ענקית שמשמעות השם באיטלקית המודרנית היא  כיכר העם  אחר כך הלכנו למלון   להשקה של המכונית    ואכלנו שם ארוחת צהרים בערב הלכנו להסתובב ברובע היהודי ובשכונת  יום אחרי זה נסענו לוותיקן  המדינה העצמאית הקטנה בעולם מבחינת שטחה  פחות ממחצית קמ ר בתוך העיר רומא שבאיטליה וכן מבחינת האוכלוסייה  כ נפשות שם נכנסנו לבזיליקת פטרוס הקדוש  מבנה הדת הנוצרי הגדול בעולם ויכול להכיל בתוכו עד כ איש בערב הלכנו לאכול 

# Prepare the training data

-Extract the story column ('text') from the training DataFrame (df_train).

-Extract the gender column ('gender') from the training DataFrame as the target (y_train).

### Apply vectorization to the training data:

A CountVectorizer object (vectorizer) is initialized.
The fit_transform method of the vectorizer is used to convert the preprocessed text data (X_train) into numerical features (X_train_features).
The fit_transform step tokenizes the text and represents it as a numerical matrix, where each row represents a document (text sample) and each column represents a unique word (token) in the text.
The values in the matrix represent the frequency of each word in the corresponding document.

In [77]:
X_train = df_train['story']
y_train = df_train['gender']
vectorizer = CountVectorizer(ngram_range=(1,3), min_df=5, max_df=400, max_features=10000)
X_train_features = vectorizer.fit_transform(X_train)

# Train and evaluate different models using cross-validation

Define a list of models:

The list contains tuples for each model, where each tuple consists of the model name, the model object, and a parameter grid for hyperparameter tuning.

In [78]:
models = [
    ('SVM', SVC(), {'kernel': ['linear', 'rbf'], 'C': [1.0, 10.0]}),
    ('Random Forest', RandomForestClassifier(), {'n_estimators': [100, 200], 'max_depth': [None, 5]}),
    ('Logistic Regression', LogisticRegression(), {'C': [1.0, 10.0], 'solver': ['liblinear']}),
    ('Naive Bayes', MultinomialNB(), {'alpha': [0.5, 1.0]}),
    ('Decision Tree', DecisionTreeClassifier(), {'max_depth': [None, 5]}),
    ('k-Nearest Neighbors', KNeighborsClassifier(), {'n_neighbors': [3, 5]}),]


#### Perform grid search cross-validation for each model:

Iterate through each model in the list.
Initialize a GridSearchCV object (grid_search) with the current model, the corresponding parameter grid, and the scoring metric.
Fit the grid search object to the training data (X_train_features, y_train).
Retrieve the best parameters and best F1 score from the grid search results.

In [79]:
f1_macro = make_scorer(f1_score, average='macro')

for model_name, model, param_grid in models:
    grid_search = GridSearchCV(model, param_grid, scoring=f1_macro, cv=10)
    grid_search.fit(X_train_features, y_train)
    best_params = grid_search.best_params_
    best_score = grid_search.best_score_
    
    print('Model:', model_name)
    print('Best Parameters:', best_params)
    print('Best F1 Score (Macro):', best_score)
    print()

GridSearchCV(cv=10, estimator=SVC(),
             param_grid={'C': [1.0, 10.0], 'kernel': ['linear', 'rbf']},
             scoring=make_scorer(f1_score, average=macro))

Model: SVM
Best Parameters: {'C': 1.0, 'kernel': 'linear'}
Best F1 Score (Macro): 0.6675090262696929



GridSearchCV(cv=10, estimator=RandomForestClassifier(),
             param_grid={'max_depth': [None, 5], 'n_estimators': [100, 200]},
             scoring=make_scorer(f1_score, average=macro))

Model: Random Forest
Best Parameters: {'max_depth': None, 'n_estimators': 100}
Best F1 Score (Macro): 0.5775532775238356



GridSearchCV(cv=10, estimator=LogisticRegression(),
             param_grid={'C': [1.0, 10.0], 'solver': ['liblinear']},
             scoring=make_scorer(f1_score, average=macro))

...

Model: Naive Bayes
Best Parameters: {'alpha': 0.5}
Best F1 Score (Macro): 0.7081567916235643



GridSearchCV(cv=10, estimator=DecisionTreeClassifier(),
             param_grid={'max_depth': [None, 5]},
             scoring=make_scorer(f1_score, average=macro))

Model: Decision Tree
Best Parameters: {'max_depth': None}
Best F1 Score (Macro): 0.6232898852235308



GridSearchCV(cv=10, estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': [3, 5]},
             scoring=make_scorer(f1_score, average=macro))

Model: k-Nearest Neighbors
Best Parameters: {'n_neighbors': 3}
Best F1 Score (Macro): 0.4937781560626028



# Apply text preprocessing to the test data
Remove any non-Hebrew characters from the text data. 

In [80]:
df_test['story'] = df_train['story'].apply(lambda x: "".join([char for char in x if char not in punctuations]))
pattern = r'[^א-ת\s]|\d'
df_test['story'] =df_train['story'].apply(lambda x: re.sub(pattern, '', x))
print(df_test.iloc[5]['story'])

לפני כ חודשים טסתי לרומא למשך שבוע טסתי במטוס אל על הטיסה ארכה כ וחצי שעות לא היו עיכובים נחתנו בערך בשעה  בבוקר ברומא מהשדה תעופה נסענו למלון  משם הלכנו למזרקת טרווי  לפי אגדה מסורתית השלכת מטבע למזרקה ביד ימין מעבר לכתף שמאל מבטיחה את שובו של המשליך לרומא מעריכים כי מטבעות בשווי כשלושת אלפים אירו נגרפות מדי לילה מתחתית המזרקה הסתובבנו באזור של המזרקה לקראת הערב נסענו לארוחת ערב במלון  יום למחרת נסענו בבוקר לגני וילה בורגוזה  גן בסגנון אנגלי בגן מצויים מספר מבנים המשמשים כיום כמוזיאונים והוא הגן הציבורי השני בגודלו ברומא השכרנו אוטו גולף והיינו שם בערך כשעתיים משם המשכנו לפיאצה דל פופולו  כיכר ענקית שמשמעות השם באיטלקית המודרנית היא  כיכר העם  אחר כך הלכנו למלון   להשקה של המכונית    ואכלנו שם ארוחת צהרים בערב הלכנו להסתובב ברובע היהודי ובשכונת  יום אחרי זה נסענו לוותיקן  המדינה העצמאית הקטנה בעולם מבחינת שטחה  פחות ממחצית קמ ר בתוך העיר רומא שבאיטליה וכן מבחינת האוכלוסייה  כ נפשות שם נכנסנו לבזיליקת פטרוס הקדוש  מבנה הדת הנוצרי הגדול בעולם ויכול להכיל בתוכו עד כ איש בערב הלכנו לאכול 

# Make predictions on the test data:

The best trained model(Naive Bayes) is used to predict the gender labels for the test data (X_test_features).
The predicted gender labels are stored in the predictions variable.

In [81]:
X_test = df_test['story']
X_train_features = vectorizer.fit_transform(X_train)

In [82]:
best_model = MultinomialNB(alpha=0.5) 
best_model.fit(X_train_features, y_train)


MultinomialNB(alpha=0.5)

In [83]:
X_test = df_test['story']
X_test_features = vectorizer.transform(X_test)
predictions = best_model.predict(X_test_features)

In [84]:
predictions_df = pd.DataFrame({'predicted_gender': predictions})
df_test = pd.concat([df_test, predictions_df], axis=1)
display(df_test.head())
display(df_test.tail())

,test_example_id,story,predicted_gender
0,0,כשחבר הזמין אותי לחול לא באמת חשבתי שזה יקרה פ...,m
1,1,לפני שהתגייסתי לצבא עשיתי כל מני מיונים ליחידו...,m
2,2,מאז שהתחילו הלימודים חלומו של כל סטודנט זה הפנ...,f
3,3,כשהייתי ילד מטוסים היה הדבר שהכי ריתק אותי בתו...,m
4,4,הייתי מדריכה בכפר נוער ומתאם הכפר היינו צריכים...,f


,test_example_id,story,predicted_gender
318,318,בחודש האחרון התחלתי תפקיד חדש בעבודה תפקיד מאו...,m
319,319,אחרי עוד סמסטר בזום ותקופת מבחנים עגומה החלטתי...,m
320,320,ב לאחר הרבה התלבטויות טסתי לבד לניו יורקחששתי ...,f
321,321,ביום הבחירות נסענו לבקר את אימי זל בבית הקברות...,m
322,322,אי שם בינואר אני בת זוגתי אחי וגיסתי סגרנו טי...,m


### Save output to csv (optional)
After you're done save your output to the 'classification_results.csv' csv file.<br/>
We assume that the dataframe with your results contain the following columns:
* column 1 (left column): 'test_example_id'  - the same id associated to each of the test stories to be predicted.
* column 2 (right column): 'predicted_category' - the predicted gender value for each of the associated story. 

Assuming your predicted values are in the `df_predicted` dataframe, you should save you're results as following:

In [85]:
df_predicted.to_csv('classification_results.csv',index=False)

NameError: name 'df_predicted' is not defined